# SQL Index Demo

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [19]:
from gpt_index import SimpleDirectoryReader, WikipediaReader
from IPython.display import Markdown, display

### Load Wikipedia Data

In [ ]:
# install wikipedia python package
!pip install wikipedia

In [2]:
wiki_docs = WikipediaReader().load_data(pages=['Toronto', 'Berlin', 'Tokyo'])

### Create Database Schema

In [3]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column

In [4]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData(bind=engine)

In [5]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all()

### Build Index

In [20]:
from gpt_index import GPTSQLStructStoreIndex, SQLDatabase
from langchain import OpenAI
from gpt_index import LLMPredictor

In [21]:
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-002"))

In [9]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [10]:
sql_database.table_info

"Table 'city_stats' has columns: city_name (VARCHAR(16)), population (INTEGER), country (VARCHAR(16))."

In [ ]:
# NOTE: the table_name specified here is the table that you
# want to extract into from unstructured documents.
index = GPTSQLStructStoreIndex(
    wiki_docs, 
    sql_database=sql_database, 
    table_name="city_stats",
    llm_predictor=llm_predictor
)

In [12]:
# view current table
stmt = select(
    [column("city_name"), column("population"), column("country")]
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)


[('Toronto', 2731571, 'Canada'), ('Berlin', 600000, 'Germany'), ('Tokyo', 13929286, 'Japan')]


### Query Index

We first show a raw SQL query, which directly executes over the table

In [15]:
response = index.query("SELECT city_name from city_stats", mode="sql")

> [query] Total LLM token usage: 0 tokens
> [query] Total embedding token usage: 0 tokens


In [14]:
display(Markdown(f"<b>{response}</b>"))

<b>[('Berlin',), ('Tokyo',), ('Toronto',)]</b>

Here we show a natural language query, which is translated to a SQL query under the hood

In [16]:
# set Logging to DEBUG for more detailed outputs
response = index.query("Which city has the highest population?", mode="default")

> Predicted SQL query: SELECT city_name, population
FROM city_stats
ORDER BY population DESC
LIMIT 1
> [query] Total LLM token usage: 144 tokens
> [query] Total embedding token usage: 0 tokens


In [23]:
display(Markdown(f"<b>{response}</b>"))

<b>[('Tokyo', 13929286)]</b>

In [18]:
# you can also fetch the raw result from SQLAlchemy! 
response.extra_info["result"]

[('Tokyo', 13929286)]

### Using Langchain for Querying

Since our SQLDatabase inherits from langchain, you can also use langchain itself for querying purposes.

In [22]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

In [24]:
llm = OpenAI(temperature=0)

In [26]:
# set Logging to DEBUG for more detailed outputs
db_chain = SQLDatabaseChain(llm=llm, database=sql_database)

In [27]:
db_chain.run("Which city has the highest population?")



> Entering new SQLDatabaseChain chain...
Which city has the highest population? 
SQLQuery: SELECT city_name FROM city_stats ORDER BY population DESC LIMIT 1;
SQLResult: [('Tokyo',)]
Answer: Tokyo has the highest population.
> Finished chain.


' Tokyo has the highest population.'